<a href="https://colab.research.google.com/github/SmritiReddyy/MLOps_OCR_Only/blob/main/Latest_ENG_SPA_Translation4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # NEW INSTALLATION CELL

# %%time
# # ============================================================
# # CLEAN + STABLE INSTALL CELL
# # ============================================================

# # First remove conflicting packages (clean state)
# !pip uninstall -y numpy paddlepaddle paddleocr torch transformers sentencepiece 2>/dev/null

# # Install stable NumPy FIRST (critical)
# !pip install numpy==1.26.4

# # --- Core Tools ---
# !pip install -q pymupdf Pillow

# # --- OCR Tools ---
# !pip install -q pytesseract
# !apt-get install -qq tesseract-ocr tesseract-ocr-spa tesseract-ocr-por > /dev/null 2>&1

# # --- PaddleOCR ---
# !pip install paddlepaddle==2.6.2
# !pip install paddleocr==2.8.1

# # --- Transformers / NLLB ---
# !pip install torch==2.2.2
# !pip install transformers==4.38.2 sentencepiece

# # --- LangChain ---
# !pip install langchain==0.0.352
# !pip install -q langchain-community

# print("\n" + "="*60)
# print("✅ All tools installed successfully!")
# print("="*60)


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: torch 2.10.0+cu128
Uninstalling torch-2.10.0+cu128:
  Successfully uninstalled torch-2.10.0+cu128
Found existing installation: transformers 5.0.0
Uninstalling transformers-5.0.0:
  Successfully uninstalled transformers-5.0.0
Found existing installation: sentencepiece 0.2.1
Uninstalling sentencepiece-0.2.1:
  Successfully uninstalled sentencepiece-0.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 131.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.1 requires torch>=1.13.0, which is not installed.
peft 0.18.1 requires transformers, which is not installed.
torchtune 0.6.1 requires sentencepiece, which is not ins

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━


✅ All tools installed successfully!
CPU times: user 13.6 s, sys: 1.89 s, total: 15.5 s
Wall time: 3min 21s


In [1]:
import os
from pathlib import Path

WORK_DIR = Path("/content/courtaccess_benchmark")
WORK_DIR.mkdir(exist_ok=True)

# ============================================================
# OPTION A: Upload your own PDF
# ============================================================
from google.colab import files
print("Upload a government form PDF (or skip to use the generated sample):")
print("(Click 'Choose Files' below, or just run the next cell to use the sample)\n")

try:
    uploaded = files.upload()
    if uploaded:
        for fname, content in uploaded.items():
            pdf_path = WORK_DIR / fname
            pdf_path.write_bytes(content)
            print(f"✅ Uploaded: {pdf_path}")
            INPUT_PDF = str(pdf_path)
except KeyboardInterrupt:
    print("Skipped upload. Will use generated sample.")
    INPUT_PDF = None

Upload a government form PDF (or skip to use the generated sample):
(Click 'Choose Files' below, or just run the next cell to use the sample)



Saving multi mix.pdf to multi mix (1).pdf
✅ Uploaded: /content/courtaccess_benchmark/multi mix (1).pdf


In [2]:
# ============================================================
# NLLB TRANSLATION SETUP
# ============================================================

import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = torch.device("cuda")
# MODEL_NAME = "facebook/nllb-200-distilled-600M"
MODEL_NAME = "facebook/nllb-200-distilled-1.3B"

print("⏳ Loading NLLB model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
print("✅ NLLB ready")

# SOURCE_LANG = "spa_Latn"
# TARGET_LANG = "eng_Latn"

SOURCE_LANG = "eng_Latn"
TARGET_LANG = "spa_Latn"

# ── Patterns that should NEVER be translated ──────────────────
SKIP_PATTERNS = [
    r'^[\d\s\.\-\/\(\)\\,]+$',          # pure numbers/dates/punctuation
    r'^\s*[xX]\s*$',                      # signature placeholder "X"
    r'^https?://',                         # URLs
    r'^\$[\d,\.]+$',                       # dollar amounts
    r'^[\d]{1,2}[\/\-][\d]{1,2}[\/\-][\d]{2,4}$',  # date formats
    r'^[\-\s]*$',
    r'^\[[\d\.\(\)a-zA-Z\;\s]+\]', # [14.1(a)] style citations
    r'^\d+\.\s*$',                  # standalone "1." "2." etc
]

def should_translate(text: str) -> bool:
    t = text.strip()
    if not t:
        return False
    # Too short to translate meaningfully
    if len(t) <= 3:
        return False
    # All-caps short label (e.g. "DATE", "NO.", "BBO NO.", "DIVISION:")
    # Allow longer all-caps like full sentences in headings
    # if t.isupper() and len(t) < 25:
    if t.isupper() and len(t) < 12:
        return False

    # Matches a skip pattern
    for pat in SKIP_PATTERNS:
        if re.match(pat, t):
            return False
    return True


def split_into_sentences(text: str) -> list[str]:
    """
    Split on sentence-ending punctuation so NLLB gets complete
    sentences rather than fragments, which reduces hallucination.
    """
    # Split on . ! ? followed by space or end-of-string
    parts = re.split(r'(?<=[.!?])\s+', text.strip())
    return [p for p in parts if p.strip()]


def _raw_batch_translate(texts: list[str]) -> list[str]:
    """Low-level call to NLLB. Input texts should all be non-empty."""
    tokenizer.src_lang = SOURCE_LANG
    encoded = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(device)

    # Limit output length relative to input to curb hallucination
    input_len = encoded["input_ids"].shape[1]
    max_out = min(512, input_len + 40)

    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(TARGET_LANG),
        max_length=max_out,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


def batch_translate(texts: list[str]) -> list[str]:
    """
    Main translation function — drop-in replacement for your original.

    Changes vs original:
      1. Skips untranslatable strings (labels, numbers, short all-caps)
      2. Splits long strings into sentences before translating
      3. Length-ratio guard: if output is suspiciously longer or shorter
         than input, falls back to the original text
    """
    results = []

    for text in texts:
        # ── Skip guard ────────────────────────────────────────
        if not should_translate(text):
            results.append(text)
            continue

        # ── Sentence splitting for long text ──────────────────
        sentences = split_into_sentences(text)

        if len(sentences) <= 1:
            # Translate as single string
            translated_list = _raw_batch_translate([text])
            translated = translated_list[0]
        else:
            # Translate sentence by sentence, then rejoin
            translated_parts = _raw_batch_translate(sentences)
            translated = " ".join(translated_parts)

        # ── Length-ratio hallucination guard ──────────────────
        ratio = len(translated) / max(len(text), 1)
        if ratio > 2.8 or ratio < 0.15:
            # Suspiciously long or short — fall back to original
            print(f"   ⚠️  Hallucination suspected (ratio={ratio:.1f}), keeping original: '{text[:60]}'")
            results.append(text)
        else:
            results.append(translated)

    return results



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


⏳ Loading NLLB model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ NLLB ready


In [3]:
# ============================================================
# UNIFIED RECONSTRUCTION — Handles Digital, Scanned & Hybrid PDFs
# ============================================================
import os
import sys
import pymupdf
from paddleocr import PaddleOCR

# Disable PaddleOCR's slow model source check
os.environ["PADDLE_PDX_DISABLE_MODEL_SOURCE_CHECK"] = "True"

# --- PaddleOCR Fix for Colab ---
# Attempt to reset PaddleX state if it was left initialized by a failed run
try:
    # We access the module directly from sys.modules if it exists to avoid triggering __init__
    if 'paddlex.repo_manager.core' in sys.modules:
        import paddlex.repo_manager.core as pdx_core
        # Try to reset internal flags if they exist (heuristics)
        if hasattr(pdx_core, '_GlobalContext'):
            pdx_core._GlobalContext._initialized = False
            print("🔄 Attempted to reset PaddleX state.")
except Exception:
    pass

PaddleOCR = None  # Will be set if import succeeds
try:
    from paddleocr import PaddleOCR
    print(f"✅ PaddleOCR: available")
except Exception as e:
    PaddleOCR = None
    print(f"⚠️ PaddleOCR unavailable: {e}")
    print(f"   Scanned pages will be skipped. RESTART RUNTIME to fix.")

OCR_DPI = 300
CONFIDENCE_THRESHOLD = 0.85

font_map = {
    "Helvetica": "helv", "Helvetica-Bold": "hebo",
    "Helvetica-Oblique": "heit", "Helvetica-BoldOblique": "hebi",
    "Times-Roman": "tiro", "Times-Bold": "tibo",
    "Times-Italic": "tiit", "Times-BoldItalic": "tibi",
    "Courier": "cour", "Courier-Bold": "cobo",
    "Courier-Oblique": "coit", "Courier-BoldOblique": "cobi",
    "Symbol": "symb", "ZapfDingbats": "zadb",
}

def get_font_code(span, font_map):
    fontname = span.get("font", "")
    flags = span.get("flags", 0)
    direct = font_map.get(fontname)
    if direct:
        return direct
    is_bold = bool(flags & 16) or "bold" in fontname.lower()
    is_italic = bool(flags & 2) or "italic" in fontname.lower() or "oblique" in fontname.lower()
    is_mono = bool(flags & 8) or "courier" in fontname.lower() or "mono" in fontname.lower()
    is_serif = bool(flags & 4) or "times" in fontname.lower() or "serif" in fontname.lower()
    if is_mono:
        if is_bold and is_italic: return "cobi"
        if is_bold: return "cobo"
        if is_italic: return "coit"
        return "cour"
    elif is_serif:
        if is_bold and is_italic: return "tibi"
        if is_bold: return "tibo"
        if is_italic: return "tiit"
        return "tiro"
    else:
        if is_bold and is_italic: return "hebi"
        if is_bold: return "hebo"
        if is_italic: return "heit"
        return "helv"

def estimate_fontsize(text, target_width, fontname="helv"):
    font = pymupdf.Font(fontname)
    best_size, best_diff = 5, float('inf')
    for s10 in range(50, 260, 2):
        size = s10 / 10.0
        w = font.text_length(text, fontsize=size)
        diff = abs(w - target_width)
        if diff < best_diff:
            best_diff = diff
            best_size = size
        if w > target_width * 1.1:
            break
    return best_size

def analyze_page(page):
    """Determine if a page is digital, scanned, or OCR'd scan."""
    td = page.get_text("dict", flags=pymupdf.TEXTFLAGS_TEXT)
    span_count = 0
    img_blocks = 0
    for b in td["blocks"]:
        if b["type"] == 1:
            img_blocks += 1
        elif b["type"] == 0:
            for line in b["lines"]:
                for span in line["spans"]:
                    if span["text"].strip():
                        span_count += 1

    images = page.get_images(full=True)
    drawings = page.get_drawings()

    # Check image coverage using multiple methods
    page_area = page.rect.width * page.rect.height
    total_img_area = 0

    # Method 1: get_image_rects (works for most PDFs)
    for img in images:
        xref = img[0]
        try:
            rects = page.get_image_rects(xref)
            for r in rects:
                total_img_area += r.width * r.height
        except:
            pass

    # Method 2: text dict image blocks (fallback)
    if total_img_area == 0:
        for b in td["blocks"]:
            if b["type"] == 1:
                bbox = b["bbox"]
                total_img_area += (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])

    # Method 3: check raw image dimensions (fallback for embedded scans)
    # If get_image_rects fails but images exist with large pixel dimensions,
    # estimate coverage from the image size vs page size
    has_large_image = False
    if total_img_area == 0 and len(images) > 0:
        for img in images:
            img_w, img_h = img[2], img[3]
            if img_w > 1000 and img_h > 1000:
                has_large_image = True
                # Estimate: a scan image this large likely fills the page
                total_img_area = page_area * 0.9
                break

    img_coverage = total_img_area / page_area if page_area > 0 else 0

    # Classification logic:
    #
    # DIGITAL: has text spans, has vector drawings (lines/borders/boxes), small images
    # SCANNED: few text spans, large image, no vector drawings
    # OCR'd SCAN: has text spans (OCR layer), large image OR no drawings, no vector art
    #
    # The KEY distinguisher: vector drawings.
    # Digital court forms always have dozens of drawn lines/borders.
    # Scanned pages have ZERO vector drawings — everything is pixels.

    is_scanned = False
    page_type = "DIGITAL"

    if span_count < 10 and len(images) > 0:
        # Classic scan: no text, has image
        is_scanned = True
        page_type = "SCANNED"
    elif img_coverage > 0.5 and len(drawings) < 5:
        # Large image covering page + almost no vector drawings = scan
        is_scanned = True
        page_type = "OCR'd SCAN" if span_count >= 10 else "SCANNED"
    elif len(drawings) < 5 and span_count > 10 and (has_large_image or len(images) > 0):
        # No vector drawings + has text + has images = OCR'd scan
        # Real digital PDFs with text ALWAYS have vector drawings for borders/lines
        is_scanned = True
        page_type = "OCR'd SCAN"

    return {
        "is_scanned": is_scanned,
        "page_type": page_type,
        "span_count": span_count,
        "images": len(images),
        "img_blocks": img_blocks,
        "drawings": len(drawings),
        "img_coverage": round(img_coverage * 100, 1),
        "has_large_image": has_large_image,
    }


def fit_fontsize(text, original_size, bbox_width, fontname="helv", min_size=4.0):
    """
    Returns a font size <= original_size such that text fits within bbox_width.
    Falls back to min_size if it still doesn't fit.
    """
    font = pymupdf.Font(fontname)
    size = original_size
    while size >= min_size:
        w = font.text_length(text, fontsize=size)
        if w <= bbox_width * 1.05:  # allow 5% overflow tolerance
            return size
        size -= 0.5
    return min_size


def fit_text_in_rect(page, rect, text, fontname="helv", color=(0, 0, 0),
                     original_size=10.0, min_size=4.0):
    """
    Insert text into a rect, scaling font down to fit width.
    If text still can't fit at min_size, truncate with ellipsis.
    """
    bbox_width = rect.width
    bbox_height = rect.height

    fitted_size = fit_fontsize(text, original_size, bbox_width, fontname, min_size)

    font = pymupdf.Font(fontname)
    # Truncate if even at min size it overflows
    if font.text_length(text, fontsize=fitted_size) > bbox_width * 1.05:
        # Truncate character by character
        for end in range(len(text), 0, -1):
            truncated = text[:end] + "…"
            if font.text_length(truncated, fontsize=fitted_size) <= bbox_width:
                text = truncated
                break

    # Vertical centering: baseline is ~0.78 of height for most fonts
    origin = pymupdf.Point(rect.x0, rect.y0 + min(rect.height * 0.78, rect.height - 1))

    try:
        page.insert_text(origin, text, fontsize=fitted_size, fontname=fontname, color=color)
    except Exception:
        pass


# ============================================================
# UPDATED: reconstruct_digital_page
# ============================================================

def reconstruct_digital_page(out_page, font_map):
    text_dict = out_page.get_text("dict", flags=pymupdf.TEXTFLAGS_TEXT)
    all_spans = []

    for block in text_dict["blocks"]:
        if block["type"] != 0:
            continue
        for line in block["lines"]:
            for span in line["spans"]:
                if span["text"].strip():
                    all_spans.append(span)

    if not all_spans:
        return 0

    # Collect texts and batch translate
    original_texts = [span["text"] for span in all_spans]
    translated_texts = batch_translate(original_texts)

    # Redact originals
    for span in all_spans:
        out_page.add_redact_annot(pymupdf.Rect(span["bbox"]), fill=(1, 1, 1))
    out_page.apply_redactions(images=pymupdf.PDF_REDACT_IMAGE_NONE)

    # Reinsert with size fitting
    count = 0
    for span, text in zip(all_spans, translated_texts):
        if not text.strip():
            continue

        bbox = pymupdf.Rect(span["bbox"])
        color = pymupdf.sRGB_to_rgb(span["color"])
        fnt = get_font_code(span, font_map)
        original_size = span["size"]

        # Scale font to fit translated text in the original bounding box
        fitted_size = fit_fontsize(text, original_size, bbox.width, fnt)

        origin = pymupdf.Point(span["origin"])
        # If fitted size is much smaller, re-anchor vertically to avoid clipping
        if fitted_size < original_size * 0.6:
            origin = pymupdf.Point(bbox.x0, bbox.y0 + bbox.height * 0.78)

        try:
            out_page.insert_text(origin, text, fontsize=fitted_size,
                                  fontname=fnt, color=color)
            count += 1
        except Exception:
            try:
                out_page.insert_text(origin, text, fontsize=fitted_size,
                                      fontname="helv", color=color)
                count += 1
            except Exception:
                pass

    return count


# ============================================================
# UPDATED: reconstruct_scanned_page
# ============================================================

def reconstruct_scanned_page(out_page, ocr_engine, page_num_for_log):
    zoom = OCR_DPI / 72
    pix = out_page.get_pixmap(matrix=pymupdf.Matrix(zoom, zoom))
    img_bytes = pix.tobytes("png")

    import tempfile, os
    tmp = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    tmp.write(img_bytes)
    tmp.close()

    result = ocr_engine.ocr(tmp.name, cls=True)
    os.unlink(tmp.name)

    detections = result[0] if result and result[0] else []
    if not detections:
        return 0

    page_w, page_h = out_page.rect.width, out_page.rect.height
    scale_x = page_w / pix.width
    scale_y = page_h / pix.height

    # Filter by confidence and build regions
    regions = []
    for det in detections:
        bbox_px = det[0]
        text = det[1][0]
        conf = det[1][1]
        # if conf < CONFIDENCE_THRESHOLD:
        if conf < 0.80 or len(text.strip()) < 3:
            continue
        xs = [p[0] for p in bbox_px]
        ys = [p[1] for p in bbox_px]
        rect = pymupdf.Rect(
            min(xs) * scale_x - 1, min(ys) * scale_y - 1,
            max(xs) * scale_x + 1, max(ys) * scale_y + 1,
        )
        regions.append({"rect": rect, "text": text})

    if not regions:
        return 0

    # White-out all regions
    shape = out_page.new_shape()
    for r in regions:
        shape.draw_rect(r["rect"])
        shape.finish(color=None, fill=(1, 1, 1))
    shape.commit(overlay=True)

    # Batch translate
    original_texts = [r["text"] for r in regions]
    translated_texts = batch_translate(original_texts)

    # Insert with size fitting
    count = 0
    for r, text in zip(regions, translated_texts):
        rect = r["rect"]
        if not text.strip():
            continue

        # Estimate font size from original box height, then fit to width
        initial_size = rect.height * 0.75
        fitted_size = fit_fontsize(text, initial_size, rect.width, "helv", min_size=4.0)
        fitted_size = max(4.0, min(fitted_size, rect.height * 0.9))

        origin = pymupdf.Point(rect.x0, rect.y0 + rect.height * 0.78)
        try:
            out_page.insert_text(origin, text, fontsize=fitted_size,
                                  fontname="helv", color=(0, 0, 0))
            count += 1
        except Exception:
            pass

    return count

    # Collect OCR texts
    original_texts = [r["text"] for r in regions]

    # Batch translate
    translated_texts = batch_translate(original_texts)

    # Insert translated text
    count = 0
    for r, text in zip(regions, translated_texts):
        rect = r["rect"]

        if not text.strip():
            continue

        fontsize = estimate_fontsize(text, rect.width)
        fontsize = max(5, min(fontsize, rect.height * 0.9))
        origin = pymupdf.Point(rect.x0, rect.y0 + rect.height * 0.78)

        try:
            out_page.insert_text(origin, text, fontsize=fontsize, fontname="helv", color=(0, 0, 0))
            count += 1
        except:
            pass



# ============================================================
# MAIN: Process all pages with auto-detection
# ============================================================
src_doc = pymupdf.open(INPUT_PDF)
out_doc = pymupdf.open()
out_doc.insert_pdf(src_doc)
src_doc.close()

# Lazy-init OCR only if needed
ocr_engine = None

digital_pages = 0
scanned_pages = 0

for pg in range(len(out_doc)):
    out_page = out_doc[pg]
    info = analyze_page(out_page)

    if not info["is_scanned"]:
        # DIGITAL PAGE
        print(f"\n📄 Page {pg+1}/{len(out_doc)}: {info['page_type']} ({info['span_count']} spans, {info['drawings']} drawings, {info['img_coverage']}% img coverage)")
        count = reconstruct_digital_page(out_page, font_map)
        print(f"   ✅ Redacted & re-inserted {count} spans")
        digital_pages += 1
    else:
        # SCANNED PAGE
        print(f"\n📷 Page {pg+1}/{len(out_doc)}: {info['page_type']} ({info['span_count']} spans, {info['images']} images → using OCR)")
        if ocr_engine is None:
            if PaddleOCR is None:
                print("   ❌ PaddleOCR not available — skipping scanned page. RESTART RUNTIME.")
                scanned_pages += 1
                continue
            print("  ⏳ Initializing PaddleOCR (first scanned page)...")
            # ocr_engine = PaddleOCR(use_angle_cls=True, lang='en')
            #Change 2
            ocr_engine = PaddleOCR(use_textline_orientation=True,lang='en')

        count = reconstruct_scanned_page(out_page, ocr_engine, pg)
        print(f"   ✅ OCR'd & inserted {count} text regions")
        scanned_pages += 1

UNIFIED_PDF = str(WORK_DIR / "unified_reconstructed.pdf")
out_doc.save(UNIFIED_PDF)
out_doc.close()

print(f"\n{'='*60}")
print(f"🎉 UNIFIED RECONSTRUCTION COMPLETE")
print(f"   Digital pages: {digital_pages}")
print(f"   Scanned pages: {scanned_pages}")
print(f"   Output: {UNIFIED_PDF}")
print(f"{'='*60}")

✅ PaddleOCR: available

📄 Page 1/3: DIGITAL (42 spans, 54 drawings, 0.4% img coverage)
   ✅ Redacted & re-inserted 42 spans

📄 Page 2/3: DIGITAL (48 spans, 60 drawings, 0.3% img coverage)
   ⚠️  Hallucination suspected (ratio=3.8), keeping original: ' discovery and all reciprocal discovery items ordered by the'
   ⚠️  Hallucination suspected (ratio=9.6), keeping original: '[14.1(a)]. '
   ⚠️  Hallucination suspected (ratio=7.1), keeping original: '    [14.1(a)]. '
   ⚠️  Hallucination suspected (ratio=3.1), keeping original: '[14(b)(1)(H); 14.1(a)]. '
   ⚠️  Hallucination suspected (ratio=10.4), keeping original: '[14(b)(1)(G)]. '
   ⚠️  Hallucination suspected (ratio=9.6), keeping original: '[14.1(a)]. '
   ⚠️  Hallucination suspected (ratio=3.1), keeping original: '[14(b)(1)(F); 14.1(a)]. '
   ✅ Redacted & re-inserted 48 spans

📷 Page 3/3: OCR'd SCAN (270 spans, 1 images → using OCR)
  ⏳ Initializing PaddleOCR (first scanned page)...
download https://paddleocr.bj.bcebos.com/PP-OCRv3/

100%|██████████| 4.00M/4.00M [00:36<00:00, 109kiB/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:21<00:00, 468kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:15<00:00, 140kiB/s]


[2026/02/21 14:57:05] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c